In [ ]:
from models.vlm import VLM
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

vlm = VLM(device=device)

conversation3 = [
    {
        "role": "<|User|>",
        "content": "<|ref|>怎样使用 deepseek-vl2<|/ref|>.",
        "images": [],    
    },
    {"role": "<|Assistant|>", "content": ""},
]

# a = vlm(conversation3, [], False, 0.4, 0.9)
# a

In [ ]:
vlm.vl_gpt.device

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import AutoModelForCausalLM

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

In [ ]:
text = '''<|system|>
你是一个分类助手，根据用户输入判断情感倾向（积极/消极）。
</s>
<|user|>
商品质量很好，但物流太慢了。
</s>
<|assistant|>
</s>
'''
inputs = tokenizer(text, return_tensors="pt")

outputs = vl_gpt.language.generate(**inputs.to(vl_gpt.device),
                                   pad_token_id=tokenizer.eos_token_id,
                                   bos_token_id=tokenizer.bos_token_id,
                                   eos_token_id=tokenizer.eos_token_id,
                                   max_new_tokens=512)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
result

In [ ]:
# conversation = [
#     {
#         "role": "User",
        
#         # "content": """<image_placeholder> is a human portrait photo, 
#         # the top dress may show or hide as the display area, 
#         # if this human have top dress what is the top dress? 
#         # as this photo what is the possible age of this human? 
#         # as this photo what is the gender of this human? 
#         # as this photo what is the facial attribute of this human? 
#         # using previous answers to form a general description of this human.
#         # as the following format 'Geometry description: ... ; Texture description: ...'
#         # """,
#         "content": """estimating the age of the people in the <image_placeholder>.""",
        
#         "images": ["dataset/celeba-pbr/masked_rgb/00002_masked_rgb.png"]
#     },
#     {
#         "role": "Assistant",
#         "content": ""
#     }
# ]

conversation1 = [
    {
        "role": "<|User|>",
        "content": "<image>\n<|ref|>图片中的人的可能的年龄是多少?<|/ref|>.",
        "images": ["dataset/celeba-pbr/masked_rgb/00020_masked_rgb.png"],
    },
    {"role": "<|Assistant|>", "content": ""},
]

conversation2 = [
    {
        "role": "<|User|>",
        "content": "<image>\n<|ref|>What's the possible age of the people in the image?<|/ref|>.",
        "images": ["dataset/celeba-pbr/masked_rgb/00020_masked_rgb.png"],
    },
    {"role": "<|Assistant|>", "content": ""},
]

conversation3 = [
    {
        "role": "<|User|>",
        "content": "<|ref|>怎样使用 deepseek-vl2<|/ref|>.",
        "images": [],    
    },
    {"role": "<|Assistant|>", "content": ""},
]

conversation = conversation3

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=[],
    force_batchify=True,
    system_prompt="你**必须**全程使用英文回答，禁止出现任何其他语言。即使用户输入包含中文词汇，你仍需用英文完整表述，并在必要时对中文术语附加英文翻译。"
)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
    temperature=0.4,
    top_p=0.9
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False)
print(f"{prepare_inputs['sft_format'][0]}", answer)

In [ ]:
prepare_inputs